# 03 – Data Preparation for Multimodal House Price Prediction

**Goal of this notebook**

In this notebook we:

1. Start from the cleaned `df_clean` table produced in the previous EDA step.
2. Define a **single target variable** for sale price and create a log-transformed version.
3. Construct:
   - A unified **text description** field for each listing.
   - A robust **image path** for at least one photo per listing (using `primary_photo` and, if needed, `alt_photos`).
4. Restrict to the subset of properties that have:
   1. A valid sale price,
   2. A non-trivial text description, and
   3. At least one downloadable photo.
5. Perform a 8/1/1 random split on this multimodal subset

The resulting train/val/test CSVs will be the *only* data used for:

- Tabular baselines, and  
- Multimodal deep learning models.

This guarantees that all models are compared on exactly the same set of properties.


In [1]:
# Imports & paths

import os
import sys
import ast
import json
from pathlib import Path

import numpy as np
import pandas as pd

# Detect whether we're running in Colab
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Colab: mount Google Drive and point PROJECT_ROOT to your repo
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive", force_remount=False)

    PROJECT_ROOT = Path("/content/drive/My Drive/SH").resolve()
    DATA_DIR = PROJECT_ROOT / "data"
else:
    # Local / non-Colab: infer paths from src/paths.py
    from src.paths import get_project_paths

    paths = get_project_paths()
    PROJECT_ROOT = paths["PROJECT_ROOT"]
    DATA_DIR = paths["DATA_DIR"]

# Ensure PROJECT_ROOT is on sys.path so "src" is importable
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Directories for processed tables and images
PROC_DIR = DATA_DIR / "processed"
IMG_DIR = DATA_DIR / "images"

PROC_DIR.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)

print("IN_COLAB    :", IN_COLAB)
print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_DIR    :", DATA_DIR)
print("PROC_DIR    :", PROC_DIR)
print("IMG_DIR     :", IMG_DIR)

Mounted at /content/drive
IN_COLAB    : True
PROJECT_ROOT: /content/drive/My Drive/SH
DATA_DIR    : /content/drive/My Drive/SH/data
PROC_DIR    : /content/drive/My Drive/SH/data/processed
IMG_DIR     : /content/drive/My Drive/SH/data/images


## 1. Load `df_clean`

We assume that `df_clean` was saved at the end of the EDA notebook into
`/content/drive/My Drive/SH/data/processed/df_clean.csv`.

If you already have `df_clean` in memory (because you are running in the same session),
you can skip the loading cell below. Otherwise, we reload it from disk.


In [2]:
# load df_clean

df_clean_path = PROC_DIR / "redfin_nj_sold_2016plus_basic_clean.csv"

if df_clean_path.exists():
    df_clean = pd.read_csv(df_clean_path)
    print("Loaded df_clean from", df_clean_path)
else:
    raise FileNotFoundError(
        f"{df_clean_path} not found. Please save df_clean from the EDA notebook first."
    )

df_clean.shape, df_clean.columns.tolist()[:20]


Loaded df_clean from /content/drive/My Drive/SH/data/processed/redfin_nj_sold_2016plus_basic_clean.csv


((179598, 58),
 ['property_id',
  'listing_id',
  'mls',
  'mls_id',
  'status',
  'mls_status',
  'text',
  'style',
  'formatted_address',
  'full_street_line',
  'street',
  'unit',
  'city',
  'state',
  'zip_code',
  'beds',
  'full_baths',
  'half_baths',
  'sqft',
  'year_built'])

In [3]:
## THIS CELL IS ONLY FOR SMALL BATCH TESTING, COMMENT FOR FINAL PROJECT SUBMISSION
#print(f"Original df_clean shape: {df_clean.shape}")

# Randomly subset 20% of the data for storage efficiency
#df_clean = df_clean.sample(frac=0.2, random_state=42).reset_index(drop=True)

#print(f"New df_clean shape (20% subset): {df_clean.shape}")

## 2. Define target variable and basic target transforms

I will model the **current sale price** using the `sold_price` column as the primary target:

- `sold_price`: price for the current sale event.  
- I create a log-transformed target `log_sold_price = log1p(sold_price)` for training stability.

Later, evaluation metrics (RMSE, MAE) will be reported in the **original price space** by exponentiating.


In [4]:
# 2. Define target and create log-transformed target

TARGET_RAW_COL = "sold_price"
TARGET_LOG_COL = "log_sold_price"

if TARGET_RAW_COL not in df_clean.columns:
    raise KeyError(f"Column {TARGET_RAW_COL} not found in df_clean.")

# Basic sanity checks on the raw target
print("sold_price summary:")
display(df_clean[TARGET_RAW_COL].describe())

# Keep only positive, non-missing sale prices
has_target = df_clean[TARGET_RAW_COL].notna() & (df_clean[TARGET_RAW_COL] > 0)
print("Rows with valid sold_price:", has_target.sum(), "of", len(df_clean))

df_clean = df_clean.loc[has_target].copy()

# Canonical transform: log1p(sold_price)
# (Modeling notebooks can back-transform using src.targets.backtransform(..., 'log1p'))
df_clean[TARGET_LOG_COL] = np.log1p(df_clean[TARGET_RAW_COL])

print("\nTarget columns summary:")
display(df_clean[[TARGET_RAW_COL, TARGET_LOG_COL]].describe())


sold_price summary:


,sold_price
count,1.795980e+05
mean,5.270593e+05
std,3.757823e+05
min,3.000000e+04
25%,2.900000e+05
50%,4.400000e+05
75%,6.500000e+05
max,3.000000e+06


Rows with valid sold_price: 179598 of 179598

Target columns summary:


,sold_price,log_sold_price
count,1.795980e+05,179598.000000
mean,5.270593e+05,12.956244
std,3.757823e+05,0.687035
min,3.000000e+04,10.308986
25%,2.900000e+05,12.577640
50%,4.400000e+05,12.994532
75%,6.500000e+05,13.384729
max,3.000000e+06,14.914123


## 3. Build a unified text description

We have two text-related columns:

- `text`: main listing description / remarks,
- `nearby_schools`: optional, often multi-line text.

We construct a single field `description_text` by concatenating:

- `text`
- `nearby_schools` (when present)

We will later require this field to be **non-null and non-trivial** (length above a small threshold) for inclusion in the multimodal subset.


In [ ]:
# construct description_text

TEXT_MAIN_COL = "text"
TEXT_FORMATED_ADDRESS_COL = "formatted_address"
TEXT_SCHOOLS_COL = "nearby_schools"
DESCRIPTION_COL = "description_text"

def make_description(row):
    parts = []
    for col in [TEXT_MAIN_COL, TEXT_FORMATED_ADDRESS_COL, TEXT_SCHOOLS_COL]:
        if col in row and pd.notna(row[col]) and str(row[col]).strip():
            parts.append(str(row[col]).strip())
    if not parts:
        return np.nan
    return " | ".join(parts)

df_clean[DESCRIPTION_COL] = df_clean.apply(make_description, axis=1)

print("description_text length stats (including NaN):")
desc_lengths = df_clean[DESCRIPTION_COL].dropna().str.len()
display(desc_lengths.describe())

description_text length stats (including NaN):


,description_text
count,179598.000000
mean,953.582935
std,537.121826
min,13.000000
25%,603.000000
50%,876.000000
75%,1193.000000
max,4274.000000


## 4. Select a photo URL per property

We want at least one photo per listing. We have:

- `primary_photo`: URL string to the main photo,
- `alt_photos`: additional photos, often represented as a list or as a stringified list.

We define a function to select **one photo URL**:

1. Use `primary_photo` if available and non-empty.
2. Otherwise, if `alt_photos` contains at least one valid URL, use the first one.

The resulting URL is stored in a new column `photo_url_selected`.


In [ ]:
# helper to parse alt_photos and select a URL

PRIMARY_PHOTO_COL = "primary_photo"
ALT_PHOTOS_COL = "alt_photos"
PHOTO_URL_COL = "photo_url_selected"

def parse_alt_photos(raw):
    """
    Parse the alt_photos column into a list of URLs.
    Handles:
      - already-a-list
      - stringified lists (ast.literal_eval)
      - NaN / None
    """
    if raw is None or (isinstance(raw, float) and np.isnan(raw)):
        return []
    if isinstance(raw, list):
        return [str(x) for x in raw if isinstance(x, str) and x.strip()]
    if isinstance(raw, str):
        s = raw.strip()
        # try to interpret as a Python literal list/dict
        if s.startswith("[") or s.startswith("{"):
            try:
                parsed = ast.literal_eval(s)
                if isinstance(parsed, list):
                    return [str(x) for x in parsed if isinstance(x, str) and x.strip()]
            except Exception:
                pass
        # fallback: treat as a single URL if it "looks like" one
        if s.startswith("http"):
            return [s]
    return []

def select_photo_url(row):
    # primary_photo first
    primary = row.get(PRIMARY_PHOTO_COL, None)
    if isinstance(primary, str) and primary.strip().startswith("http"):
        return primary.strip()
    # fallback to first alt photo
    alt = row.get(ALT_PHOTOS_COL, None)
    alt_list = parse_alt_photos(alt)
    if alt_list:
        return alt_list[0]
    return np.nan

df_clean[PHOTO_URL_COL] = df_clean.apply(select_photo_url, axis=1)

print("Photo URL availability:")
print(df_clean[PHOTO_URL_COL].notna().value_counts())
df_clean[[PRIMARY_PHOTO_COL, ALT_PHOTOS_COL, PHOTO_URL_COL]].head()


Photo URL availability:
photo_url_selected
True     31722
False     4198
Name: count, dtype: int64


,primary_photo,alt_photos,photo_url_selected
0,https://ap.rdcpix.com/dda40e2f4ece3e635be0753b...,https://ap.rdcpix.com/dda40e2f4ece3e635be0753b...,https://ap.rdcpix.com/dda40e2f4ece3e635be0753b...
1,https://ap.rdcpix.com/1121447753/9b44aac0656ed...,https://ap.rdcpix.com/1121447753/9b44aac0656ed...,https://ap.rdcpix.com/1121447753/9b44aac0656ed...
2,NaN,NaN,NaN
3,https://ap.rdcpix.com/f18a3a7e32d38f249be79ddf...,https://ap.rdcpix.com/f18a3a7e32d38f249be79ddf...,https://ap.rdcpix.com/f18a3a7e32d38f249be79ddf...
4,https://ap.rdcpix.com/61aedce672655a3f95e9f8f0...,https://ap.rdcpix.com/61aedce672655a3f95e9f8f0...,https://ap.rdcpix.com/61aedce672655a3f95e9f8f0...


## 5. Download photos and construct `image_path`

We now download each selected `photo_url_selected` into `data/images/` and store a filesystem path in a new column `image_path`.

- Filenames are based on `listing_id` when available, otherwise on the row index.
- If a download fails, we leave `image_path` as NaN for that row.


In [ ]:
# Code cell: download selected photo URLs -> image_path

import requests
from PIL import Image
from io import BytesIO
from tqdm.auto import tqdm

IMG_COL = "image_path"

def download_photo(row):
    url = row[PHOTO_URL_COL]
    if not isinstance(url, str) or not url.startswith("http"):
        return np.nan

    # Prefer listing_id for stable filenames
    listing_id = row.get("listing_id", None)
    if pd.notna(listing_id):
        fname = f"{int(listing_id)}.jpg"
    else:
        fname = f"idx_{int(row.name)}.jpg"

    fpath = IMG_DIR / fname

    # If file already exists, reuse it
    if fpath.exists():
        return str(fpath)

    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        img = Image.open(BytesIO(resp.content)).convert("RGB")
        img.save(fpath, format="JPEG", quality=90)
        return str(fpath)
    except Exception as e:
        # You can log e if desired
        return np.nan

# Only attempt downloads where we have a selected URL
mask_has_photo_url = df_clean[PHOTO_URL_COL].notna()
print("Rows with selected photo URL:", mask_has_photo_url.sum())

df_clean[IMG_COL] = np.nan  # initialize

for idx in tqdm(df_clean[mask_has_photo_url].index, desc="Downloading photos"):
    df_clean.at[idx, IMG_COL] = download_photo(df_clean.loc[idx])

print("Rows with non-null image_path after download:",
      df_clean[IMG_COL].notna().sum())


Rows with selected photo URL: 31722


/tmp/ipython-input-4289294595.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '/content/drive/My Drive/SH/data/images/2971447519.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_clean.at[idx, IMG_COL] = download_photo(df_clean.loc[idx])


Rows with non-null image_path after download: 31720


## 6. Define the multimodal subset

We now systematically define the subset of properties that meet **all three** criteria:

1. Valid sale price (`sold_price` > 0; already enforced above).
2. Non-trivial text description (`description_text` length > threshold, here 30 characters).
3. At least one photo downloaded successfully (`image_path` non-null).

We also explicitly report how many properties remain and their distribution across years.


In [ ]:
# 1) target is already valid by construction of df_clean above

# 2) non-trivial text description
TEXT_MIN_LENGTH = 30
has_text = (
    df_clean[DESCRIPTION_COL].notna()
    & df_clean[DESCRIPTION_COL].str.len().gt(TEXT_MIN_LENGTH)
)

# 3) at least one downloadable photo
#has_image = df_clean[IMG_COL].notna()

# multimodal mask
mm_mask = has_text #& has_image

print("Total rows in df_clean:", len(df_clean))
print("Rows with non-trivial text description:", has_text.sum())
#print("Rows with image_path:", has_image.sum())
print("Rows in multimodal subset (all three criteria):", mm_mask.sum())

df_mm = df_clean.loc[mm_mask].copy()
df_mm.shape


Total rows in df_clean: 179598
Rows with non-trivial text description: 179554
Rows in multimodal subset (all three criteria): 179554


(179554, 60)

## 7. Train/validation/test split on the multimodal subset

All downstream models (baselines and multimodal DL) will *only* use these splits of `df_mm`.


In [ ]:
np.random.seed(42)

train_frac = 0.8
val_frac = 0.1
test_frac = 0.1

# Shuffle and split
df_mm_shuffled = df_mm.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(len(df_mm_shuffled) * train_frac)
val_size = int(len(df_mm_shuffled) * val_frac)

df_mm_train = df_mm_shuffled.iloc[:train_size].copy()
df_mm_val = df_mm_shuffled.iloc[train_size : train_size + val_size].copy()
df_mm_test = df_mm_shuffled.iloc[train_size + val_size :].copy()

print("Multimodal subset sizes (random split):")
print("  Train:", len(df_mm_train))
print("  Val  :", len(df_mm_val))
print("  Test :", len(df_mm_test))

# Safety check: ensure total rows match
assert len(df_mm_train) + len(df_mm_val) + len(df_mm_test) == len(df_mm)

Multimodal subset sizes (random split):
  Train: 143643
  Val  : 17955
  Test : 17956


## 8. Select columns for modeling and save CSVs

For modeling, I keep these columns:

- Target:
  - `sold_price` (raw),  
  - `log_sold_price` (log transform used for training).
- Text:
  - `description_text`
- Image:
  - `image_path`
- Tabular features (numeric + categorical):
  - numeric: `beds`, `full_baths`, `half_baths`, `sqft`, `year_built`, `days_on_mls`, `lot_sqft`, `price_per_sqft`, `hoa_fee`, `fed_funds_rate`, etc.
  - categorical: `city`, `state`, `zip_code`, `status`, `style`, `parking_garage`, `new_construction`, `stories`, `county`, etc.

You can refine this list later; here we start with a reasonable set and only keep columns that actually exist in `df_mm`.


In [ ]:
TARGET_RAW_COL = "sold_price"
TARGET_LOG_COL = "log_sold_price"
TEXT_COL = DESCRIPTION_COL
#IMG_PATH_COL = IMG_COL

TAB_NUMERIC = [
    "beds",
    "full_baths",
    "half_baths",
    "sqft",
    "year_built",
    "days_on_mls",
    "lot_sqft",
    #"price_per_sqft",
    "hoa_fee",
    "fed_funds_rate",
]

TAB_CATEGORICAL = [
    "city",
    "state",
    "zip_code",
    "status",
    "style",
    "parking_garage",
    "new_construction",
    "stories",
    "county",
]

# keep only columns that exist in df_mm
TAB_NUMERIC = [c for c in TAB_NUMERIC if c in df_mm.columns]
TAB_CATEGORICAL = [c for c in TAB_CATEGORICAL if c in df_mm.columns]

META_COLS = ["sale_year", "sale_date", "property_id", "listing_id", "mls", "mls_id"]
META_COLS = [c for c in META_COLS if c in df_mm.columns]

KEEP_COLS = (
    [TARGET_RAW_COL, TARGET_LOG_COL, TEXT_COL,
     #IMG_PATH_COL
     ]
    + TAB_NUMERIC
    + TAB_CATEGORICAL
    + META_COLS
)

KEEP_COLS = [c for c in KEEP_COLS if c in df_mm.columns]

print("Numeric tabular features:", TAB_NUMERIC)
print("Categorical tabular features:", TAB_CATEGORICAL)
print("Meta columns:", META_COLS)
print("Total kept columns:", len(KEEP_COLS))

train_path = PROC_DIR / "train_multimodal.csv"
val_path   = PROC_DIR / "val_multimodal.csv"
test_path  = PROC_DIR / "test_multimodal.csv"

df_mm_train[KEEP_COLS].to_csv(train_path, index=False)
df_mm_val[KEEP_COLS].to_csv(val_path, index=False)
df_mm_test[KEEP_COLS].to_csv(test_path, index=False)

train_path, val_path, test_path

Numeric tabular features: ['beds', 'full_baths', 'half_baths', 'sqft', 'year_built', 'days_on_mls', 'lot_sqft', 'hoa_fee', 'fed_funds_rate']
Categorical tabular features: ['city', 'state', 'zip_code', 'status', 'style', 'parking_garage', 'new_construction', 'stories', 'county']
Meta columns: ['sale_year', 'sale_date', 'property_id', 'listing_id', 'mls', 'mls_id']
Total kept columns: 27


(PosixPath('/content/drive/My Drive/SH/data/processed/train_multimodal.csv'),
 PosixPath('/content/drive/My Drive/SH/data/processed/val_multimodal.csv'),
 PosixPath('/content/drive/My Drive/SH/data/processed/test_multimodal.csv'))

In [ ]:
df_mm_val.shape

(17955, 60)

## 9. Save a small JSON summary for reporting

Finally, we save a small JSON file summarizing:

- Overall counts in the original `df_clean`,
- Counts after each filter step,
- Final train/val/test sizes for the multimodal subset.

This will be useful to reference in the project report.


In [ ]:
# Code cell: summary JSON

summary = {
    "total_clean_rows": int(len(df_clean)),
    "multimodal_rows_total": int(len(df_mm)),
    "splits": {
        "train": int(len(df_mm_train)),
        "val":   int(len(df_mm_val)),
        "test":  int(len(df_mm_test)),
    },
    "criteria": {
        "target_column": TARGET_RAW_COL,
        "log_target_column": TARGET_LOG_COL,
        "text_min_length": TEXT_MIN_LENGTH,
        "split_type": "random",
        "train_frac": train_frac,
        "val_frac": val_frac,
        "test_frac": test_frac,
        "random_state": 42,
        "numeric_features": TAB_NUMERIC,
        "categorical_features": TAB_CATEGORICAL,
    },
}

summary_path = PROC_DIR / "multimodal_prep_summary.json"
with open(summary_path, "w") as f:
    json.dump(summary, f, indent=2)

summary_path, summary

(PosixPath('/content/drive/My Drive/SH/data/processed/multimodal_prep_summary.json'),
 {'total_clean_rows': 179598,
  'multimodal_rows_total': 179554,
  'splits': {'train': 143643, 'val': 17955, 'test': 17956},
  'criteria': {'target_column': 'sold_price',
   'log_target_column': 'log_sold_price',
   'text_min_length': 30,
   'split_type': 'random',
   'train_frac': 0.8,
   'val_frac': 0.1,
   'test_frac': 0.1,
   'random_state': 42,
   'numeric_features': ['beds',
    'full_baths',
    'half_baths',
    'sqft',
    'year_built',
    'days_on_mls',
    'lot_sqft',
    'hoa_fee',
    'fed_funds_rate'],
   'categorical_features': ['city',
    'state',
    'zip_code',
    'status',
    'style',
    'parking_garage',
    'new_construction',
    'stories',
    'county']}})